<a href="https://colab.research.google.com/github/mohitraosatya/IQM_Optimization/blob/main/IQM_Geneopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Quantum Circuit Optimization using Nature-Inspired Genetic Algorithms

Author: Satya Mohit Rao Kamkanampati

Email: saka4331@colorado.edu

This notebook explores a nature-inspired approach to optimizing quantum circuits using Genetic Algorithms (GAs), taking inspiration from biological evolution. The goal is to maximize the probability of a desired output state (e.g., `'00'`) in a 2-qubit quantum system simulated via Qiskit. By evolving gate parameters over generations, we demonstrate how GAs can effectively search the quantum parameter space and improve circuit fidelity.

This work aligns with the ongoing challenges in quantum computing—particularly in error mitigation and hardware-efficient circuit design—and offers a potential direction for scaling intelligent quantum compilers on real hardware like IQM’s superconducting quantum processors.


In [4]:
!pip install numpy qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.5 MB/s eta 0:00:00


In [5]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

In [6]:
def create_quantum_circuit(params):
    qc = QuantumCircuit(2)
    qc.rx(params[0], 0)
    qc.ry(params[1], 1)
    qc.cx(0, 1)
    qc.measure_all()
    return qc

In [7]:
def fitness_function(params):
    qc = create_quantum_circuit(params)
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit).result()
    counts = result.get_counts()
    # Objective: Maximize the probability of measuring '00'
    return counts.get('00', 0) / sum(counts.values())

In [8]:
def initialize_population(size):
    return np.random.uniform(0, 2 * np.pi, (size, 2))

In [9]:
def select_parents(population, fitnesses, num_parents):
    parents = population[np.argsort(fitnesses)][-num_parents:]
    return parents

In [10]:
def crossover(parents, offspring_size):
    offspring = np.empty((offspring_size, parents.shape[1]))
    crossover_point = np.uint8(parents.shape[1]/2)
    for k in range(offspring_size):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [11]:
def mutation(offspring_crossover, mutation_rate=0.1):
    for idx in range(offspring_crossover.shape[0]):
        if np.random.rand() < mutation_rate:
            random_index = np.random.randint(0, offspring_crossover.shape[1])
            random_value = np.random.uniform(0, 2 * np.pi)
            offspring_crossover[idx, random_index] = random_value
    return offspring_crossover

In [12]:
def genetic_algorithm(pop_size, num_generations, num_parents):
    population = initialize_population(pop_size)
    for generation in range(num_generations):
        fitnesses = np.array([fitness_function(ind) for ind in population])
        parents = select_parents(population, fitnesses, num_parents)
        offspring_crossover = crossover(parents, offspring_size=pop_size - num_parents)
        offspring_mutation = mutation(offspring_crossover)
        population[:num_parents] = parents
        population[num_parents:] = offspring_mutation
        best_fitness = np.max(fitnesses)
        print(f"Generation {generation}: Best Fitness = {best_fitness}")
    best_solution = population[np.argmax(fitnesses)]
    return best_solution

In [16]:
best_params = genetic_algorithm(pop_size=20, num_generations=10, num_parents=10)
print("Optimized Parameters:", best_params)

Generation 0: Best Fitness = 0.84765625
Generation 1: Best Fitness = 0.8408203125
Generation 2: Best Fitness = 0.896484375
Generation 3: Best Fitness = 0.91015625
Generation 4: Best Fitness = 0.89453125
Generation 5: Best Fitness = 0.9248046875
Generation 6: Best Fitness = 0.92578125
Generation 7: Best Fitness = 0.9189453125
Generation 8: Best Fitness = 0.9990234375
Generation 9: Best Fitness = 0.9921875
Optimized Parameters: [0.14470787 0.02442485]
